## Lineare und polynomiale Regression für Kürbispreise - Lektion 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografik von Dasani Madipalli</figcaption>


#### Einführung

Bisher hast du untersucht, was Regression ist, anhand von Beispieldaten aus dem Kürbispreis-Datensatz, den wir in dieser Lektion verwenden werden. Du hast sie auch mit `ggplot2` visualisiert. 💪

Jetzt bist du bereit, tiefer in die Regression für maschinelles Lernen einzutauchen. In dieser Lektion wirst du mehr über zwei Arten der Regression lernen: *einfache lineare Regression* und *polynomiale Regression*, zusammen mit einigen mathematischen Grundlagen dieser Techniken.

> Im gesamten Lehrplan gehen wir von minimalen mathematischen Kenntnissen aus und versuchen, das Thema für Studierende aus anderen Bereichen zugänglich zu machen. Achte daher auf Hinweise, 🧮 Erläuterungen, Diagramme und andere Lernhilfen, die das Verständnis erleichtern.

#### Vorbereitung

Zur Erinnerung: Du lädst diese Daten, um Fragen dazu zu stellen.

-   Wann ist der beste Zeitpunkt, Kürbisse zu kaufen?

-   Welchen Preis kann ich für eine Kiste mit Miniaturkürbissen erwarten?

-   Sollte ich sie in halben Scheffelkörben oder in einer 1 1/9 Scheffelbox kaufen? Lass uns weiter in diese Daten eintauchen.

In der vorherigen Lektion hast du ein `tibble` (eine moderne Neuinterpretation des Dataframes) erstellt und es mit einem Teil des ursprünglichen Datensatzes gefüllt, wobei du die Preise standardisiert nach Scheffel angegeben hast. Dadurch konntest du jedoch nur etwa 400 Datenpunkte sammeln, und das nur für die Herbstmonate. Vielleicht können wir durch eine gründlichere Bereinigung der Daten mehr Details über die Natur der Daten erhalten? Mal sehen... 🕵️‍♀️

Für diese Aufgabe benötigen wir die folgenden Pakete:

-   `tidyverse`: Das [tidyverse](https://www.tidyverse.org/) ist eine [Sammlung von R-Paketen](https://www.tidyverse.org/packages), die Datenwissenschaft schneller, einfacher und unterhaltsamer macht!

-   `tidymodels`: Das [tidymodels](https://www.tidymodels.org/) Framework ist eine [Sammlung von Paketen](https://www.tidymodels.org/packages/) für Modellierung und maschinelles Lernen.

-   `janitor`: Das [janitor-Paket](https://github.com/sfirke/janitor) bietet einfache kleine Werkzeuge zur Untersuchung und Bereinigung von unordentlichen Daten.

-   `corrplot`: Das [corrplot-Paket](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) bietet ein visuelles Explorationswerkzeug für Korrelationsmatrizen, das die automatische Neuordnung von Variablen unterstützt, um versteckte Muster zwischen Variablen zu erkennen.

Du kannst sie wie folgt installieren:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Das untenstehende Skript überprüft, ob du die für dieses Modul benötigten Pakete hast, und installiert sie für dich, falls sie fehlen.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Wir werden später diese großartigen Pakete laden und sie in unserer aktuellen R-Sitzung verfügbar machen. (Das dient nur zur Veranschaulichung, `pacman::p_load()` hat das bereits für Sie erledigt.)

## 1. Eine lineare Regressionslinie

Wie Sie in Lektion 1 gelernt haben, besteht das Ziel einer linearen Regression darin, eine *Linie* *der* *besten Anpassung* zu zeichnen, um:

-   **Variable Beziehungen zu zeigen**. Die Beziehung zwischen Variablen darzustellen.

-   **Vorhersagen zu treffen**. Präzise Vorhersagen darüber zu machen, wo ein neuer Datenpunkt im Verhältnis zu dieser Linie liegen würde.

Um diese Art von Linie zu zeichnen, verwenden wir eine statistische Technik namens **Least-Squares Regression**. Der Begriff `least-squares` bedeutet, dass alle Datenpunkte um die Regressionslinie herum quadriert und dann addiert werden. Idealerweise ist diese endgültige Summe so klein wie möglich, da wir eine geringe Anzahl von Fehlern oder `least-squares` anstreben. Die Linie der besten Anpassung ist daher die Linie, die uns den niedrigsten Wert für die Summe der quadrierten Fehler liefert – daher der Name *Least-Squares Regression*.

Wir tun dies, weil wir eine Linie modellieren möchten, die die geringste kumulative Entfernung von all unseren Datenpunkten hat. Wir quadrieren die Terme vor dem Addieren, da uns die Größe und nicht die Richtung interessiert.

> **🧮 Zeig mir die Mathematik**
>
> Diese Linie, genannt *Linie der besten Anpassung*, kann durch [eine Gleichung](https://en.wikipedia.org/wiki/Simple_linear_regression) ausgedrückt werden:
>
>     Y = a + bX
>
> `X` ist die '`erklärende Variable` oder `Prädiktor`'. `Y` ist die '`abhängige Variable` oder `Ergebnis`'. Die Steigung der Linie ist `b` und `a` ist der y-Achsenabschnitt, der den Wert von `Y` angibt, wenn `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "Steigung = $y/x$")
    Infografik von Jen Looper
>
> Zuerst berechnen wir die Steigung `b`.
>
> Anders ausgedrückt, und bezogen auf die ursprüngliche Frage zu unseren Kürbisdaten: "den Preis eines Kürbisses pro Scheffel nach Monat vorhersagen", würde `X` den Preis und `Y` den Verkaufsmonat darstellen.
>
> ![](../../../../../../translated_images/calculation.989aa7822020d9d0ba9fc781f1ab5192f3421be86ebb88026528aef33c37b0d8.de.png)
    Infografik von Jen Looper
> 
> Berechnen Sie den Wert von Y. Wenn Sie etwa 4 \$ zahlen, muss es April sein!
>
> Die Mathematik, die die Linie berechnet, muss die Steigung der Linie demonstrieren, die auch vom Achsenabschnitt abhängt, oder wo `Y` liegt, wenn `X = 0`.
>
> Sie können die Methode zur Berechnung dieser Werte auf der Website [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html) beobachten. Besuchen Sie auch [diesen Least-Squares-Rechner](https://www.mathsisfun.com/data/least-squares-calculator.html), um zu sehen, wie die Werte der Zahlen die Linie beeinflussen.

Gar nicht so beängstigend, oder? 🤓

#### Korrelation

Ein weiterer Begriff, den Sie verstehen sollten, ist der **Korrelationskoeffizient** zwischen den gegebenen X- und Y-Variablen. Mithilfe eines Streudiagramms können Sie diesen Koeffizienten schnell visualisieren. Ein Diagramm mit Datenpunkten, die in einer ordentlichen Linie angeordnet sind, hat eine hohe Korrelation, während ein Diagramm mit Datenpunkten, die überall zwischen X und Y verstreut sind, eine niedrige Korrelation aufweist.

Ein gutes lineares Regressionsmodell ist eines, das einen hohen (näher an 1 als an 0) Korrelationskoeffizienten mit der Least-Squares Regression-Methode und einer Regressionslinie aufweist.


## **2. Ein Tanz mit Daten: Erstellen eines Dataframes für das Modellieren**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Kunstwerk von @allison_horst</figcaption>


<!--![Kunstwerk von \@allison_horst](../../../../../../translated_images/janitor.e4a77dd3d3e6a32e25327090b8a9c00dc7cf459c44fa9f184c5ecb0d48ce3794.de.jpg){width="700"}-->


Lade die benötigten Bibliotheken und den Datensatz. Konvertiere die Daten in einen Dataframe, der einen Teil der Daten enthält:

-   Nur Kürbisse auswählen, die nach Scheffelpreis bewertet sind

-   Das Datum in einen Monat umwandeln

-   Den Preis als Durchschnitt aus hohen und niedrigen Preisen berechnen

-   Den Preis so umrechnen, dass er die Bewertung nach Scheffelmenge widerspiegelt

> Diese Schritte haben wir in der [vorherigen Lektion](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb) behandelt.


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

Im Geiste des reinen Abenteuers lassen Sie uns das [`janitor package`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) erkunden, das einfache Funktionen zum Untersuchen und Bereinigen von unordentlichen Daten bietet. Zum Beispiel werfen wir einen Blick auf die Spaltennamen unserer Daten:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Wir können es besser machen. Lassen Sie uns diese Spaltennamen `friendR` machen, indem wir sie mit der [snake_case](https://de.wikipedia.org/wiki/Snake_case)-Konvention mit `janitor::clean_names` umwandeln. Um mehr über diese Funktion zu erfahren: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Viel tidyR 🧹! Jetzt ein Tanz mit den Daten, wie in der vorherigen Lektion, mit `dplyr`! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Gute Arbeit! 👌 Du hast jetzt einen sauberen, aufgeräumten Datensatz, mit dem du dein neues Regressionsmodell erstellen kannst!

Wie wäre es mit einem Streudiagramm?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Ein Streudiagramm erinnert uns daran, dass wir nur Monatsdaten von August bis Dezember haben. Wir benötigen wahrscheinlich mehr Daten, um Schlussfolgerungen auf eine lineare Weise ziehen zu können.

Werfen wir erneut einen Blick auf unsere Modellierungsdaten:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Was wäre, wenn wir den `Preis` eines Kürbisses basierend auf den Spalten `Stadt` oder `Paket`, die vom Typ Zeichenkette sind, vorhersagen wollten? Oder noch einfacher, wie könnten wir die Korrelation (die erfordert, dass beide Eingaben numerisch sind) zwischen beispielsweise `Paket` und `Preis` finden? 🤷🤷

Maschinelle Lernmodelle funktionieren am besten mit numerischen Merkmalen anstelle von Textwerten, daher müssen kategorische Merkmale in der Regel in numerische Darstellungen umgewandelt werden.

Das bedeutet, dass wir einen Weg finden müssen, unsere Prädiktoren so umzuformatieren, dass sie für ein Modell effektiver nutzbar sind – ein Prozess, der als `Feature Engineering` bekannt ist.


## 3. Vorverarbeitung von Daten für die Modellierung mit Recipes 👩‍🍳👨‍🍳

Aktivitäten, die Prädiktorwerte umformatieren, um sie für ein Modell effektiver nutzbar zu machen, werden als `Feature Engineering` bezeichnet.

Verschiedene Modelle haben unterschiedliche Anforderungen an die Vorverarbeitung. Zum Beispiel erfordert die Methode der kleinsten Quadrate die `Kodierung kategorischer Variablen` wie Monat, Sorte und Stadtname. Dies bedeutet einfach, dass eine Spalte mit `kategorischen Werten` in eine oder mehrere `numerische Spalten` übersetzt wird, die die ursprüngliche Spalte ersetzen.

Angenommen, Ihre Daten enthalten die folgende kategorische Variable:

|  Stadt   |
|:--------:|
| Denver   |
| Nairobi  |
|  Tokio   |

Sie können *ordinale Kodierung* anwenden, um jeder Kategorie einen eindeutigen Ganzzahlwert zuzuweisen, wie folgt:

| Stadt |
|:-----:|
|   0   |
|   1   |
|   2   |

Und genau das werden wir mit unseren Daten machen!

In diesem Abschnitt werden wir ein weiteres großartiges Tidymodels-Paket erkunden: [recipes](https://tidymodels.github.io/recipes/) - das entwickelt wurde, um Ihnen bei der Vorverarbeitung Ihrer Daten **vor** dem Training Ihres Modells zu helfen. Im Kern ist ein Recipe ein Objekt, das definiert, welche Schritte auf einen Datensatz angewendet werden sollen, um ihn für die Modellierung vorzubereiten.

Nun erstellen wir ein Recipe, das unsere Daten für die Modellierung vorbereitet, indem es für alle Beobachtungen in den Prädiktorspalten einen eindeutigen Ganzzahlwert ersetzt:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Super! 👏 Wir haben gerade unser erstes Rezept erstellt, das ein Ergebnis (Preis) und die dazugehörigen Prädiktoren spezifiziert und alle Prädiktorspalten in eine Menge von Ganzzahlen kodiert 🙌! Lass uns das schnell aufschlüsseln:

-   Der Aufruf von `recipe()` mit einer Formel teilt dem Rezept die *Rollen* der Variablen mit, wobei die Daten aus `new_pumpkins` als Referenz verwendet werden. Zum Beispiel wurde der `price`-Spalte die Rolle `outcome` zugewiesen, während die restlichen Spalten die Rolle `predictor` erhalten haben.

-   `step_integer(all_predictors(), zero_based = TRUE)` gibt an, dass alle Prädiktoren in eine Menge von Ganzzahlen umgewandelt werden sollen, wobei die Nummerierung bei 0 beginnt.

Wir sind sicher, dass du dir gerade Gedanken machst wie: "Das ist so cool!! Aber was, wenn ich überprüfen möchte, ob die Rezepte genau das tun, was ich von ihnen erwarte? 🤔"

Das ist ein großartiger Gedanke! Siehst du, sobald dein Rezept definiert ist, kannst du die Parameter schätzen, die erforderlich sind, um die Daten tatsächlich vorzubereiten, und dann die verarbeiteten Daten extrahieren. Normalerweise musst du das nicht tun, wenn du Tidymodels verwendest (wir werden gleich die übliche Vorgehensweise sehen -> `workflows`), aber es kann nützlich sein, wenn du eine Art Plausibilitätsprüfung durchführen möchtest, um sicherzustellen, dass die Rezepte das tun, was du erwartest.

Dafür brauchst du zwei weitere Verben: `prep()` und `bake()`. Und wie immer helfen dir unsere kleinen R-Freunde von [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) dabei, das besser zu verstehen!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Kunstwerk von @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): schätzt die erforderlichen Parameter aus einem Trainingssatz, die später auf andere Datensätze angewendet werden können. Zum Beispiel, für eine gegebene Prädiktorspalte, welche Beobachtung wird der Ganzzahl 0, 1, 2 usw. zugewiesen.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): nimmt ein vorbereitetes Rezept und wendet die Operationen auf jeden beliebigen Datensatz an.

Das gesagt, lassen Sie uns unsere Rezepte vorbereiten und anwenden, um wirklich zu bestätigen, dass die Prädiktorspalten im Hintergrund zuerst kodiert werden, bevor ein Modell angepasst wird.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Woo-hoo!🥳 Die verarbeiteten Daten `baked_pumpkins` haben alle ihre Prädiktoren kodiert, was bestätigt, dass die im Rezept definierten Vorverarbeitungsschritte wie erwartet funktionieren. Das macht es zwar schwieriger für dich zu lesen, aber viel verständlicher für Tidymodels! Nimm dir etwas Zeit, um herauszufinden, welche Beobachtung einer entsprechenden Ganzzahl zugeordnet wurde.

Es ist auch erwähnenswert, dass `baked_pumpkins` ein Dataframe ist, auf dem wir Berechnungen durchführen können.

Zum Beispiel können wir versuchen, eine gute Korrelation zwischen zwei Punkten deiner Daten zu finden, um möglicherweise ein gutes Vorhersagemodell zu erstellen. Dafür verwenden wir die Funktion `cor()`. Gib `?cor()` ein, um mehr über die Funktion zu erfahren.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Wie sich herausstellt, gibt es nur eine schwache Korrelation zwischen der Stadt und dem Preis. Allerdings gibt es eine etwas stärkere Korrelation zwischen dem Paket und seinem Preis. Das ergibt Sinn, oder? Normalerweise gilt: Je größer die Obst- oder Gemüsekiste, desto höher der Preis.

Während wir schon dabei sind, können wir auch versuchen, eine Korrelationsmatrix aller Spalten mit dem `corrplot`-Paket zu visualisieren.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Viel besser.

Eine gute Frage, die man jetzt zu diesen Daten stellen könnte, wäre: '`Welchen Preis kann ich für ein bestimmtes Kürbispaket erwarten?`' Lassen Sie uns direkt loslegen!

> Hinweis: Wenn Sie das vorbereitete Rezept **`pumpkins_prep`** mit **`new_data = NULL`** **`bake()`**, extrahieren Sie die verarbeiteten (d.h. kodierten) Trainingsdaten. Wenn Sie ein anderes Datenset hätten, beispielsweise ein Testset, und sehen möchten, wie ein Rezept es vorverarbeiten würde, könnten Sie einfach **`pumpkins_prep`** mit **`new_data = test_set`** backen.

## 4. Erstellen Sie ein lineares Regressionsmodell

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografik von Dasani Madipalli</figcaption>


<!--![Infografik von Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.de.png){width="800"}-->


Jetzt, da wir ein Rezept erstellt und tatsächlich bestätigt haben, dass die Daten angemessen vorverarbeitet werden, bauen wir nun ein Regressionsmodell, um die folgende Frage zu beantworten: `Welchen Preis kann ich für ein bestimmtes Kürbispaket erwarten?`

#### Trainiere ein lineares Regressionsmodell mit dem Trainingsdatensatz

Wie Sie wahrscheinlich schon herausgefunden haben, ist die Spalte *price* die `Zielvariable`, während die Spalte *package* die `Prädiktorvariable` ist.

Um dies zu tun, werden wir die Daten zunächst so aufteilen, dass 80 % in den Trainings- und 20 % in den Testdatensatz gehen. Anschließend definieren wir ein Rezept, das die Prädiktorspalte in eine Reihe von Ganzzahlen kodiert, und erstellen eine Modellspezifikation. Wir werden unser Rezept nicht vorbereiten und backen, da wir bereits wissen, dass es die Daten wie erwartet vorverarbeitet.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Gut gemacht! Jetzt, da wir ein Rezept und eine Modellspezifikation haben, müssen wir eine Möglichkeit finden, diese zusammen in einem Objekt zu bündeln, das zunächst die Daten vorverarbeitet (prep+bake im Hintergrund), das Modell auf den vorverarbeiteten Daten anpasst und auch potenzielle Nachbearbeitungsaktivitäten ermöglicht. Klingt das nicht beruhigend?🤩

In Tidymodels wird dieses praktische Objekt [`workflow`](https://workflows.tidymodels.org/) genannt und enthält bequem alle Modellierungskomponenten! Das ist das, was wir in *Python* als *Pipelines* bezeichnen würden.

Also, lass uns alles in einem Workflow bündeln!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

Obendrein kann ein Workflow auf ähnliche Weise angepasst/trainiert werden wie ein Modell.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Aus den Modellergebnissen können wir die während des Trainings gelernten Koeffizienten sehen. Sie repräsentieren die Koeffizienten der Linie der besten Anpassung, die uns den geringsten Gesamtfehler zwischen der tatsächlichen und der vorhergesagten Variablen liefert.

#### Modellleistung mit dem Testdatensatz bewerten

Es ist Zeit herauszufinden, wie das Modell abgeschnitten hat 📏! Wie machen wir das?

Nachdem wir das Modell trainiert haben, können wir es verwenden, um Vorhersagen für den `test_set` mit `parsnip::predict()` zu treffen. Anschließend können wir diese Vorhersagen mit den tatsächlichen Label-Werten vergleichen, um zu bewerten, wie gut (oder nicht!) das Modell funktioniert.

Beginnen wir damit, Vorhersagen für den Testdatensatz zu machen und die Spalten an den Testdatensatz anzufügen.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Ja, Sie haben gerade ein Modell trainiert und verwendet, um Vorhersagen zu treffen! 🔮 Ist es gut? Lassen Sie uns die Leistung des Modells bewerten!

In Tidymodels machen wir das mit `yardstick::metrics()`! Für die lineare Regression konzentrieren wir uns auf die folgenden Metriken:

-   `Root Mean Square Error (RMSE)`: Die Quadratwurzel des [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Dies ergibt eine absolute Metrik in derselben Einheit wie das Label (in diesem Fall der Preis eines Kürbisses). Je kleiner der Wert, desto besser das Modell (vereinfacht gesagt repräsentiert es den durchschnittlichen Betrag, um den die Vorhersagen falsch sind!).

-   `Coefficient of Determination (üblicherweise bekannt als R-squared oder R2)`: Eine relative Metrik, bei der ein höherer Wert eine bessere Anpassung des Modells bedeutet. Im Wesentlichen repräsentiert diese Metrik, wie viel der Varianz zwischen vorhergesagten und tatsächlichen Label-Werten das Modell erklären kann.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Da geht die Modellleistung. Mal sehen, ob wir eine bessere Einschätzung bekommen, indem wir ein Streudiagramm der Pakete und Preise visualisieren und dann die Vorhersagen verwenden, um eine Linie der besten Anpassung darüber zu legen.

Das bedeutet, dass wir den Testdatensatz vorbereiten und verarbeiten müssen, um die Paketspalte zu kodieren und diese dann mit den Vorhersagen unseres Modells zu verbinden.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Wie Sie sehen können, generalisiert das lineare Regressionsmodell die Beziehung zwischen einem Paket und seinem entsprechenden Preis nicht besonders gut.

🎃 Herzlichen Glückwunsch, Sie haben gerade ein Modell erstellt, das helfen kann, den Preis einiger Kürbissorten vorherzusagen. Ihr Kürbisfeld für die Feiertage wird wunderschön sein. Aber Sie können wahrscheinlich ein besseres Modell erstellen!

## 5. Erstellen Sie ein polynomiales Regressionsmodell

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografik von Dasani Madipalli</figcaption>


<!--![Infografik von Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.de.png){width="800"}-->


Manchmal haben unsere Daten keine lineare Beziehung, aber wir möchten trotzdem ein Ergebnis vorhersagen. Die polynomiale Regression kann uns helfen, Vorhersagen für komplexere nicht-lineare Beziehungen zu treffen.

Nehmen wir zum Beispiel die Beziehung zwischen Verpackung und Preis in unserem Kürbis-Datensatz. Während es manchmal eine lineare Beziehung zwischen Variablen gibt – je größer der Kürbis im Volumen, desto höher der Preis – können diese Beziehungen manchmal nicht als Ebene oder gerade Linie dargestellt werden.

> ✅ Hier sind [einige weitere Beispiele](https://online.stat.psu.edu/stat501/lesson/9/9.8) für Daten, die polynomiale Regression verwenden könnten.
>
> Werfen Sie einen weiteren Blick auf die Beziehung zwischen Sorte und Preis im vorherigen Diagramm. Sieht dieser Streudiagramm so aus, als sollte er unbedingt mit einer geraden Linie analysiert werden? Vielleicht nicht. In diesem Fall können Sie polynomiale Regression ausprobieren.
>
> ✅ Polynome sind mathematische Ausdrücke, die aus einer oder mehreren Variablen und Koeffizienten bestehen können.

#### Trainieren eines polynomialen Regressionsmodells mit dem Trainingssatz

Die polynomiale Regression erstellt eine *gekrümmte Linie*, um nichtlineare Daten besser anzupassen.

Schauen wir, ob ein polynomiales Modell besser darin ist, Vorhersagen zu treffen. Wir folgen einem ähnlichen Verfahren wie zuvor:

-   Erstellen Sie ein Rezept, das die Vorverarbeitungsschritte angibt, die an unseren Daten durchgeführt werden müssen, um sie für die Modellierung vorzubereiten, z. B.: Kodierung von Prädiktoren und Berechnung von Polynomen eines Grades *n*.

-   Erstellen Sie eine Modellspezifikation.

-   Bündeln Sie das Rezept und die Modellspezifikation in einen Workflow.

-   Erstellen Sie ein Modell, indem Sie den Workflow anpassen.

-   Bewerten Sie, wie gut das Modell auf den Testdaten abschneidet.

Legen wir los!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Modellleistung bewerten

👏👏Du hast ein Polynommodell erstellt – lass uns Vorhersagen für den Testdatensatz machen!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, lass uns bewerten, wie das Modell auf dem test_set mit `yardstick::metrics()` abgeschnitten hat.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Viel bessere Leistung.

Das `rmse` sank von etwa 7 auf etwa 3, was darauf hinweist, dass der Fehler zwischen dem tatsächlichen Preis und dem vorhergesagten Preis reduziert wurde. Man kann dies *grob* so interpretieren, dass falsche Vorhersagen im Durchschnitt um etwa 3 \$ danebenliegen. Das `rsq` stieg von etwa 0,4 auf 0,8.

Alle diese Metriken zeigen, dass das polynomiale Modell deutlich besser abschneidet als das lineare Modell. Gute Arbeit!

Lass uns sehen, ob wir das visualisieren können!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Du kannst eine geschwungene Linie sehen, die besser zu deinen Daten passt! 🤩

Du kannst dies noch glatter machen, indem du eine polynomiale Formel an `geom_smooth` übergibst, wie hier:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Ganz wie eine geschmeidige Kurve!🤩

So erstellen Sie eine neue Vorhersage:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Die Vorhersage des `polynomial model` ergibt Sinn, wenn man die Streudiagramme von `price` und `package` betrachtet! Und falls dies ein besseres Modell als das vorherige ist, basierend auf denselben Daten, solltest du für diese teureren Kürbisse ein höheres Budget einplanen!

🏆 Gut gemacht! Du hast in einer Lektion zwei Regressionsmodelle erstellt. Im letzten Abschnitt über Regression wirst du etwas über logistische Regression lernen, um Kategorien zu bestimmen.

## **🚀Challenge**

Teste mehrere verschiedene Variablen in diesem Notebook, um zu sehen, wie die Korrelation mit der Modellgenauigkeit zusammenhängt.

## [**Post-lecture quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Review & Selbststudium**

In dieser Lektion haben wir über Lineare Regression gelernt. Es gibt andere wichtige Arten der Regression. Lies über Stepwise-, Ridge-, Lasso- und Elasticnet-Techniken. Ein guter Kurs, um mehr darüber zu lernen, ist der [Stanford Statistical Learning course](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Wenn du mehr darüber erfahren möchtest, wie du das großartige Tidymodels-Framework nutzen kannst, sieh dir bitte die folgenden Ressourcen an:

-   Tidymodels-Website: [Get started with Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn und Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **DANK AN:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) für die Erstellung der großartigen Illustrationen, die R einladender und ansprechender machen. Weitere Illustrationen findest du in ihrer [Galerie](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Haftungsausschluss**:  
Dieses Dokument wurde mithilfe des KI-Übersetzungsdienstes [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, weisen wir darauf hin, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die aus der Nutzung dieser Übersetzung entstehen.
